In [87]:
import pandas as pd
import numpy as np
import sklearn 
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
import pandas_datareader as dr

In [180]:
# parameter configuration
start = '2018-09-27'
end = '2019-11-23'
param_list = ['Close']
lags = [1,2,3,4,5,-1,-2,-3,-4,-5]
future_lags = [-1,-2,-3,-4,-5]
autocorrelation_col = 'Close'
autocorrelation_lags = [i for i in range(1,20)]
#lags = [i for i in range (20)]

In [97]:
spsymbols = pd.read_csv('spsymbols.csv')
spsymbols = spsymbols['Symbol'].values.tolist()

In [146]:
# index of apple stock
counter = 0
for i in spsymbols:
    if i == 'AAPL':
        print(counter)
    counter += 1

51


In [147]:
def get_data(spsymbols, start, end):
    dict_of_dfs = {}
    for symbol in spsymbols[48:53]:
        print(symbol)
        data = dr.data.get_data_yahoo(symbol, start, end)
        dict_of_dfs[symbol] = data
    return dict_of_dfs

In [150]:
data = get_data(spsymbols, start='2005-09-27', end='2019-11-23')

AON
APA
AIV
AAPL
AMAT


In [151]:
data['AAPL'].shape

(3565, 6)

In [178]:
def add_features(df, param_list, lags, create_target=True, target_col='High', target_name='next_week_max_price', agg='max'):
    max_val = max(lags)
    min_val = min(lags)
    
    future_lags = []
    for param in param_list:
        for lag in lags:
            if lag > 0:
                new_feature = param + '_t-' + str(lag)
            if lag < 0:
                new_feature = param + '_t+' + str(abs(lag))
                future_lags.append(new_feature)
                
            df[new_feature] = df[param].shift(lag)
            
    if create_target:
        if agg == 'max':
            df[target_name] = df[future_lags].max(axis=1)  
            
    if min_val < 0: 
        df = df.iloc[max_val:min_val,:]
    else:
        df = df.iloc[max_val:,:]
        
    return df

In [114]:
df = add_features(aapl_data, param_list, lags)
df.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume,Close_t-1,Close_t-2,Close_t-3,Close_t-4,Close_t-5,Close_t+1,Close_t+2,Close_t+3,Close_t+4,Close_t+5,next_week_max_price
5,1980-12-19,0.504464,0.506696,0.504464,0.504464,0.400656,12157600.0,0.475446,0.462054,0.450893,0.486607,0.513393,0.529018,0.551339,0.580357,0.633929,0.642857,0.642857
6,1980-12-22,0.529018,0.531250,0.529018,0.529018,0.420157,9340800.0,0.504464,0.475446,0.462054,0.450893,0.486607,0.551339,0.580357,0.633929,0.642857,0.627232,0.642857
7,1980-12-23,0.551339,0.553571,0.551339,0.551339,0.437885,11737600.0,0.529018,0.504464,0.475446,0.462054,0.450893,0.580357,0.633929,0.642857,0.627232,0.609375,0.642857
8,1980-12-24,0.580357,0.582589,0.580357,0.580357,0.460931,12000800.0,0.551339,0.529018,0.504464,0.475446,0.462054,0.633929,0.642857,0.627232,0.609375,0.616071,0.642857
9,1980-12-26,0.633929,0.636161,0.633929,0.633929,0.503479,13893600.0,0.580357,0.551339,0.529018,0.504464,0.475446,0.642857,0.627232,0.609375,0.616071,0.602679,0.642857
10,1980-12-29,0.642857,0.645089,0.642857,0.642857,0.510570,23290400.0,0.633929,0.580357,0.551339,0.529018,0.504464,0.627232,0.609375,0.616071,0.602679,0.575893,0.627232
11,1980-12-30,0.629464,0.629464,0.627232,0.627232,0.498160,17220000.0,0.642857,0.633929,0.580357,0.551339,0.529018,0.609375,0.616071,0.602679,0.575893,0.551339,0.616071
12,1980-12-31,0.611607,0.611607,0.609375,0.609375,0.483978,8937600.0,0.627232,0.642857,0.633929,0.580357,0.551339,0.616071,0.602679,0.575893,0.551339,0.540179,0.616071
13,1981-01-02,0.616071,0.620536,0.616071,0.616071,0.489296,5415200.0,0.609375,0.627232,0.642857,0.633929,0.580357,0.602679,0.575893,0.551339,0.540179,0.569196,0.602679
14,1981-01-05,0.604911,0.604911,0.602679,0.602679,0.478659,8932000.0,0.616071,0.609375,0.627232,0.642857,0.633929,0.575893,0.551339,0.540179,0.569196,0.564732,0.575893


In [122]:
idxs = random.sample(range(-9800, 9800), 10)

In [127]:
t = pd.DataFrame(idxs, columns=['val'])

In [140]:
def find_autocorrelated_lags(df, autocorrelation_col, autocorrelation_lags):
    correlated_lags = []
    corr_vals = []
    col_list = df[autocorrelation_col].values.tolist() 
    s = pd.Series(col_list)
    for i in autocorrelation_lags:
        corr = s.autocorr(lag=i)
        #print(corr)
        if abs(corr) >= 0.6:
            correlated_lags.append(i)
            corr_vals.append(corr)
    return correlated_lags, corr_vals

In [153]:
#correlated_lags, corr_vals = find_autocorrelated_lags(t, 'val', [1,2,3,4,5])
correlated_lags, corr_vals = find_autocorrelated_lags(df, autocorrelation_col, autocorrelation_lags)

In [154]:
correlated_lags = correlated_lags + future_lags

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [181]:
# apple data
df = data['AAPL']
correlated_lags = correlated_lags + future_lags
df = add_features(df, param_list, correlated_lags, create_target=True, target_col='High', target_name='next_week_max_price', agg='max')

In [184]:
df.tail()

,High,Low,Open,Close,Volume,Adj Close,Close_t-1,Close_t-2,Close_t-3,Close_t-4,...,Close_t-16,Close_t-17,Close_t-18,Close_t-19,next_week_max_price,Close_t+1,Close_t+2,Close_t+3,Close_t+4,Close_t+5
Date,,,,,,,,,,,,,,,,,,,,,
2019-11-11,262.470001,258.279999,258.299988,262.200012,20455300.0,262.200012,260.140015,259.429993,257.239990,257.130005,...,236.410004,235.279999,234.369995,235.320007,267.100006,261.959991,264.470001,262.640015,265.760010,267.100006
2019-11-12,262.790009,260.920013,261.549988,261.959991,21847200.0,261.959991,262.200012,260.140015,259.429993,257.239990,...,240.509995,236.410004,235.279999,234.369995,267.100006,264.470001,262.640015,265.760010,267.100006,266.290009
2019-11-13,264.779999,261.070007,261.130005,264.470001,25683600.0,264.470001,261.959991,262.200012,260.140015,259.429993,...,239.960007,240.509995,236.410004,235.279999,267.100006,262.640015,265.760010,267.100006,266.290009,263.190002
2019-11-14,264.880005,262.100006,263.750000,262.640015,22295700.0,262.640015,264.470001,261.959991,262.200012,260.140015,...,243.179993,239.960007,240.509995,236.410004,267.100006,265.760010,267.100006,266.290009,263.190002,262.010010
2019-11-15,265.779999,263.010010,263.679993,265.760010,25051600.0,265.760010,262.640015,264.470001,261.959991,262.200012,...,243.580002,243.179993,239.960007,240.509995,267.100006,267.100006,266.290009,263.190002,262.010010,261.779999


In [185]:
cols = data_train.columns
not_relevant_cols = ["Date","Adj Close","High_t+1", "High_t+2","High_t+3", "High_t+4", "High_t+5", "High", "Volume", "Low", "Close","Open"]
relevant_cols = [i for i in cols if i not in not_relevant_cols]
ready_train_df = df[relevant_cols]

KeyError: "['Volume_t-1', 'open_t-1', 'Volume_t-3', 'Volume_t-2', 'Closeh_t-5', 'High_t-2', 'open_t-3', 'Low_t-5', 'open_t-4', 'Low_t-4', 'Low_t-1', 'Low_t-2', 'open_t-5', 'Volume_t-5', 'Volume_t-4', 'open_t-2', 'High_t-5', 'High_t-4', 'High_t-1', 'Low_t-3', 'High_t-3'] not in index"

In [28]:
cols = data_train.columns
not_relevant_cols = ["Date","Adj Close","High_t+1", "High_t+2","High_t+3", "High_t+4", "High_t+5", "High", "Volume", "Low", "Close","Open"]
relevant_cols = [i for i in cols if i not in not_relevant_cols]
ready_train_df = data_train[relevant_cols]

In [29]:
ready_train_df.shape

(9817, 26)

In [57]:
ready_train_df.head()

,High_t-1,High_t-2,High_t-3,High_t-4,High_t-5,open_t-1,open_t-2,open_t-3,open_t-4,open_t-5,...,Close_t-2,Close_t-3,Close_t-4,Closeh_t-5,Volume_t-1,Volume_t-2,Volume_t-3,Volume_t-4,Volume_t-5,next_week_max_price
5,0.477679,0.464286,0.453125,0.488839,0.515625,0.475446,0.462054,0.453125,0.488839,0.513393,...,0.462054,0.450893,0.486607,0.513393,18362400.0,21610400.0,26432000.0,43971200.0,117258400.0,0.645089
6,0.506696,0.477679,0.464286,0.453125,0.488839,0.504464,0.475446,0.462054,0.453125,0.488839,...,0.475446,0.462054,0.450893,0.486607,12157600.0,18362400.0,21610400.0,26432000.0,43971200.0,0.645089
7,0.531250,0.506696,0.477679,0.464286,0.453125,0.529018,0.504464,0.475446,0.462054,0.453125,...,0.504464,0.475446,0.462054,0.450893,9340800.0,12157600.0,18362400.0,21610400.0,26432000.0,0.645089
8,0.553571,0.531250,0.506696,0.477679,0.464286,0.551339,0.529018,0.504464,0.475446,0.462054,...,0.529018,0.504464,0.475446,0.462054,11737600.0,9340800.0,12157600.0,18362400.0,21610400.0,0.645089
9,0.582589,0.553571,0.531250,0.506696,0.477679,0.580357,0.551339,0.529018,0.504464,0.475446,...,0.551339,0.529018,0.504464,0.475446,12000800.0,11737600.0,9340800.0,12157600.0,18362400.0,0.645089


In [50]:
ready_train_df.columns

Index(['High_t-1', 'High_t-2', 'High_t-3', 'High_t-4', 'High_t-5', 'open_t-1',
       'open_t-2', 'open_t-3', 'open_t-4', 'open_t-5', 'Low_t-1', 'Low_t-2',
       'Low_t-3', 'Low_t-4', 'Low_t-5', 'Close_t-1', 'Close_t-2', 'Close_t-3',
       'Close_t-4', 'Closeh_t-5', 'Volume_t-1', 'Volume_t-2', 'Volume_t-3',
       'Volume_t-4', 'Volume_t-5', 'next_week_max_price'],
      dtype='object')

In [38]:
import random
ready_train_df.reset_index()
ready_train_df = ready_train_df.iloc[:, 1:]
#idxs = random.sample(range(0, 9800), 1200)
#not_idxs = list([i for i in range(ready_train_df.shape[0]) if i not in idxs])
#train_x = ready_train_df.iloc[idxs,:-1]
#train_y = ready_train_df.iloc[idxs,-1:]
#test_x = ready_train_df.iloc[not_idxs,:-1]
#test_y = ready_train_df.iloc[not_idxs,-1:]
train_x = ready_train_df.iloc[0:9800,:-1]
train_y = ready_train_df.iloc[0:9800,-1:]
test_x = ready_train_df.iloc[9800:,:-1]
test_y = ready_train_df.iloc[9800:,-1:]

In [39]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(9800, 23)
(9800, 1)
(17, 23)
(17, 1)


In [40]:
# normalize data
scaler = MinMaxScaler()
normalized_train = scaler.fit_transform(train_x.values)
y_train = train_y.values
normalized_test = scaler.transform(test_x.values)
y_test = test_y.values

In [41]:
# train model
model = xgb.XGBRegressor()
model.fit(normalized_train,y_train)

[11:28:06] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [42]:
# predict test
preds = model.predict(normalized_test)

In [47]:
res_tab = pd.DataFrame()
actual = [ val[0] for val in test_y.values.tolist()]
res_tab['actual'] = actual
res_tab['predicted'] = preds
res_tab['error'] = res_tab['actual'] - res_tab['predicted']
res_tab['abs_error'] = abs(res_tab['actual'] - res_tab['predicted'])
res_tab['abs_error'] = abs(res_tab['actual'] - res_tab['predicted'])
res_tab['pct_error'] = (res_tab['abs_error'] /res_tab['actual']) * 100
#res_tab['percent_error'] = abs(res_tab['actual'] - res_tab['predicted']) / res_tab['actual']

In [48]:
res_tab.head(10)

,actual,predicted,error,abs_error,pct_error
0,260.350006,254.668076,5.681930,5.681930,2.182420
1,260.440002,254.644028,5.795974,5.795974,2.225455
2,262.470001,253.370407,9.099594,9.099594,3.466908
3,262.790009,254.206955,8.583054,8.583054,3.266126
4,264.779999,254.576569,10.203430,10.203430,3.853550
5,264.880005,255.095963,9.784042,9.784042,3.693764
6,265.779999,254.977631,10.802368,10.802368,4.064402
7,267.429993,256.825562,10.604431,10.604431,3.965311
8,268.000000,256.482605,11.517395,11.517395,4.297535
9,268.000000,256.660370,11.339630,11.339630,4.231205


In [ ]:
# rmse 